In [2]:
import numpy as np
import re
from itertools import combinations
from tqdm import tqdm
import pandas as pd
from collections import Counter
from matplotlib import pyplot as plt
%matplotlib inline
from scipy import stats

In [30]:
def xipai(x):
    """
    'xipai' means shuffling the decks in chinese
    """
    for _ in range(100):
        np.random.shuffle(x)
    return x

def initialization():
    """
    This function is to initialize the decks everytime we start playing
    """
    original_state = []
    for ii in np.arange(1,10):
        for jj in ['a','b','c','d']:
            original_state.append('{}{}'.format(ii,jj))
    original_state = np.array(original_state,dtype=str)
    return original_state

def check(play):
    """
    This function is to check the cards of one single player and get all possible bull and base combinations
    if there exist these bulls and bases, we save them, 
    but if there does not exist any bull or base, we return an empty list
    """
    bases,bulls = [],[]
    for t in combinations(play,3):
        if np.sum([int(re.findall('\d+',item)[0]) for item in t]) % 10 == 0:
            bases_ = [ii for ii in t]
            bulls_ = [ii for ii in play if ii not in list(t)]
            if len(bulls_) > 0:
                bulls.append(bulls_)
                bases.append(bases_)
    return bulls,bases

def get_points(bull):
    """
    if there exists any bulls and bases, we calculate the points
    if the sum of the bull cards is greater than 10, we subtract 10
    if there does not exist any bull or base, point is 0
    """
    bulls,bases = check(bull)
    if len(bull) > 0:
        points = np.sum(int(re.findall('\d+',ii)[0]) for ii in bull) 
        if points > 10:
            points = points - 10
    else:
        points = 0
        
    return points

def get_bull_base(play):
    """
    since whichever combination of bulls and bases does not matter, we just pick one if there are more than one
    or, that is empty
    """
    bulls,bases = check(play)
    if len(bases) < 1:
        bull,base = [],[]
    elif len(bases) > 1:
        pick = np.random.choice(len(bases),size=1)[0]
        bull,base = bulls[pick],bases[pick]
    else:
        bull,base = bulls[0],bases[0]
    return bull,base

def win_lose(players,zhuang='player1',xian='player2'):
    """
    To determine winning or losing, we need to compare 2 players: zhuang and xian
    each round, we have one zhuang and the others are xian
    if zhuang wins, zhuang wins against individual xian
    but if xian wins, xian wins the points xian gets
    """
    letter2num = {'a':4,'b':3,'c':2,'d':1}
    bull1,base1 = get_bull_base(players[zhuang])
    bull2,base2 = get_bull_base(players[xian])
    #print(bull1,bull2)
    
    point1 = np.sum(int(re.findall('\d+',ii)[0]) for ii in bull1)
    point2 = np.sum(int(re.findall('\d+',ii)[0]) for ii in bull2)
    
    if point1 > point2:
        return -(point1 - point2)
    elif point1 < point2:
        return point2 - point1
    elif point1 == point2:
        cards1 = [int(re.findall('\d+',ii)[0]) for ii in players[zhuang]]
        color1 = [ii[1] for ii in players[zhuang]]
        cards2 = [int(re.findall('\d+',ii)[0]) for ii in players[xian]]
        color2 = [ii[1] for ii in players[xian]]
        
        if np.max(cards1) > np.max(cards2):
            return -(np.max(cards1) - np.max(cards2))
        elif np.max(cards1) < np.max(cards2):
            return np.max(cards2) - np.max(cards1)
        elif np.max(cards1) == np.max(cards2):
            idx1 = np.argmax(cards1)
            idx2 = np.argmax(cards2)
            
            c1 = color1[idx1]
            c2 = color2[idx2]
            
            if letter2num[c1] > letter2num[c2]:
                return -np.max(cards1)
            elif letter2num[c1] < letter2num[c2]:
                return np.max(cards2)

# Let's play the game 100 times and play these 100 times over 1000 nights

In [42]:
wins = {'player{}'.format(ii):[] for ii in np.arange(1,8)}
being = {'player{}'.format(ii):[] for ii in np.arange(1,8)}
for sim in tqdm(range(int(1e1))):
    zhuang = 'player1'
    next_zhuang = 'player1'
    cc = {'player{}'.format(ii):[] for ii in np.arange(1,8)}
    cc['zhuang']=[]
    for round_ in range(int(1e2)):
        original_state = initialization()

        shuffled_state = xipai(original_state)
        left = shuffled_state[-1]
        shuffled_state = shuffled_state[:-1].reshape(-1,5)
        shuffled_state
        players = {'player{}'.format(ii+1):row for ii,row in enumerate(shuffled_state)}

        #print(players,'\nleft out: {}'.format(left))



        [cc[player].append(get_bull_base(row)) for player,row in players.items()]
        for player,play in players.items():
            bull,base = get_bull_base(play)
            point = get_points(bull)
            if point == 10:
                next_zhuang = player
        cc['zhuang'].append(zhuang)
        zhuang = next_zhuang

        

100%|██████████████████████████████████████████| 10/10 [00:03<00:00,  3.37it/s]


In [44]:
cc = pd.DataFrame(cc)

In [49]:
cc.columns

Index(['player1', 'player2', 'player3', 'player4', 'player5', 'player6',
       'player7', 'zhuang'],
      dtype='object')

In [68]:
results = {'player{}'.format(ii):[] for ii in np.arange(1,8)}
results['zhuang'] = []
for ii, row in cc.iterrows():
    zhuang = row[row['zhuang']]
    xian = [row[player] for player in cc.columns[:-1] if (player != row['zhuang'])]
    
    point_z = get_points(zhuang[0])
    
    if point_z == 10:
        results[row['zhuang']].append(6 * 10)
        [results[player].append(-10) for player in cc.columns[:-1] if (player != row['zhuang'])]
        results['zhuang'].append(row['zhuang'])
    elif point_z == 0:
        scores = [0 - get_points(row[player][0]) for player in cc.columns[:-1] if (player != row['zhuang'])]
        results[row['zhuang']].append(np.sum(scores))
        [results[player].append(get_points(row[player][0])) for player in cc.columns[:-1] if (player != row['zhuang'])]
        results['zhuang'].append(row['zhuang'])
    else:
        scores = [point_z - get_points(row[player][0]) for player in cc.columns[:-1] if (player != row['zhuang'])]
        results[row['zhuang']].append(np.sum(scores))
        [results[player].append(get_points(row[player][0])) for player in cc.columns[:-1] if (player != row['zhuang'])]
        results['zhuang'].append(row['zhuang'])

In [72]:
cc

,player1,player2,player3,player4,player5,player6,player7,zhuang
0,"([], [])","([], [])","([], [])","([4a, 5a], [1d, 3a, 6a])","([], [])","([], [])","([], [])",player1
1,"([], [])","([], [])","([], [])","([], [])","([9c, 1a], [6a, 6b, 8a])","([], [])","([3d, 7a], [4a, 4c, 2a])",player1
2,"([8d, 2b], [9b, 4a, 7c])","([], [])","([8c, 1c], [3d, 5c, 2c])","([4c, 6a], [3c, 5d, 2d])","([], [])","([6c, 7a], [5a, 2a, 3a])","([], [])",player7
3,"([5a, 1b], [9c, 3b, 8b])","([8d, 7c], [1a, 6a, 3c])","([], [])","([], [])","([9b, 3a], [4b, 5c, 1c])","([], [])","([], [])",player4
4,"([], [])","([], [])","([], [])","([], [])","([], [])","([], [])","([5c, 9d], [2b, 1c, 7d])",player4
5,"([], [])","([3a, 7c], [4b, 5c, 1b])","([2d, 9c], [9b, 8b, 3b])","([1a, 4d], [4a, 9a, 7b])","([9d, 4c], [3c, 1c, 6a])","([2a, 6b], [5b, 7a, 8d])","([8c, 8a], [2c, 6d, 2b])",player4
6,"([6c, 3d], [8b, 8d, 4b])","([7c, 5c], [6b, 5d, 9a])","([4a, 3a], [6d, 5b, 9b])","([], [])","([6a, 1b], [2b, 1a, 7b])","([3c, 4d], [7a, 1c, 2a])","([4c, 1d], [9c, 8a, 3b])",player2
7,"([], [])","([5a, 1c], [8c, 8a, 4d])","([5d, 6b], [3a, 4b, 3d])","([7b, 8b], [6d, 9d, 5c])","([8d, 1b], [9c, 9b, 2d])","([3c, 1d], [4a, 7c, 9a])","([3b, 6c], [1a, 7a, 2c])",player2
8,"([4d, 7b], [3c, 5a, 2a])","([7a, 6d], [3d, 5d, 2c])","([6b, 4c], [2d, 7d, 1a])","([9a, 9c], [1c, 4b, 5b])","([], [])","([2b, 1b], [4a, 5c, 1d])","([8b, 3a], [6c, 6a, 8c])",player2
9,"([5d, 6b], [9a, 8c, 3a])","([], [])","([], [])","([3c, 4c], [1d, 2d, 7a])","([1a, 8d], [9d, 5b, 6c])","([9b, 5c], [2b, 7b, 1c])","([1b, 9c], [5a, 3b, 2a])",player3


In [71]:
pd.DataFrame(results)

,player1,player2,player3,player4,player5,player6,player7,zhuang
0,-9,0,0,9,0,0,0,player1
1,-20,0,0,0,10,0,10,player1
2,10,0,9,10,0,3,-32,player7
3,6,5,0,-13,2,0,0,player4
4,0,0,0,-4,0,0,4,player4
5,0,10,1,2,3,8,6,player4
6,9,-23,7,0,7,7,5,player2
7,0,8,1,5,9,4,9,player2
8,1,-5,10,8,0,3,1,player2
9,1,0,-31,7,9,4,10,player3


# After the simulation, we could correlate the number of winnings and the number of being zhuang, and it should be somewhat consistent

In [35]:
for player in wins.keys():
    print(player,stats.spearmanr(wins[player],being[player]))

player1 SpearmanrResult(correlation=0.38905954795524134, pvalue=0.2664779212005221)
player2 SpearmanrResult(correlation=-0.08510677611520905, pvalue=0.8151715445415288)
player3 SpearmanrResult(correlation=0.7112494861056756, pvalue=0.021090925985548795)
player4 SpearmanrResult(correlation=0.8997002046464955, pvalue=0.00039167359562948393)
player5 SpearmanrResult(correlation=0.7963562622208846, pvalue=0.005836997302932195)
player6 SpearmanrResult(correlation=0.790277206784084, pvalue=0.006514223587240812)
player7 SpearmanrResult(correlation=0.9636363636363635, pvalue=7.320974809529922e-06)


In [36]:
pd.DataFrame(cc)

,player1,player2,player3,player4,player5,player6,player7,zhuang
0,"([3d, 4b], [2b, 6a, 2c])","([2a, 2d], [4c, 8d, 8c])","([], [])","([5c, 6c], [5b, 7a, 8a])","([], [])","([], [])","([], [])",player1
1,"([], [])","([3c, 1d], [4c, 8d, 8a])","([1a, 1b], [9c, 9b, 2c])","([], [])","([7d, 3b], [7a, 9d, 4b])","([], [])","([], [])",player1
2,"([7a, 3c], [7c, 9a, 4a])","([1d, 4d], [4b, 7d, 9d])","([], [])","([3a, 8b], [2c, 2d, 6d])","([8a, 4c], [9b, 3b, 8d])","([], [])","([1a, 2a], [8c, 6c, 6a])",player5
3,"([], [])","([5c, 3d], [4d, 7c, 9a])","([9d, 4c], [1a, 7d, 2d])","([7a, 8c], [3b, 6d, 1d])","([5b, 9c], [1c, 7b, 2a])","([8d, 2c], [3a, 1b, 6b])","([3c, 2b], [8b, 6a, 6c])",player1
4,"([], [])","([8b, 9a], [1b, 7d, 2a])","([], [])","([], [])","([5b, 1d], [7a, 6b, 7c])","([4d, 4c], [9d, 8d, 3a])","([2d, 7b], [5d, 9b, 6c])",player6
5,"([2a, 7d], [4b, 5b, 1c])","([7a, 8d], [5d, 3b, 2c])","([7b, 8b], [1a, 3a, 6b])","([2d, 9a], [9c, 6a, 5a])","([8a, 6d], [9b, 4d, 7c])","([3c, 2b], [6c, 1b, 3d])","([4c, 8c], [5c, 4a, 1d])",player6
6,"([9b, 1b], [2b, 4b, 4d])","([8c, 5b], [6b, 3c, 1c])","([], [])","([7d, 8a], [5c, 8d, 7b])","([9d, 7a], [6d, 9a, 5a])","([], [])","([2a, 7c], [3d, 4c, 3b])",player6
7,"([3b, 5b], [4d, 7b, 9b])","([], [])","([7a, 1c], [9d, 6d, 5a])","([], [])","([], [])","([3a, 7c], [8d, 6c, 6a])","([8b, 2a], [1a, 2c, 7d])",player1
8,"([8d, 8c], [4d, 9d, 7d])","([], [])","([4a, 8a], [5a, 2a, 3a])","([2d, 4b], [3b, 6a, 1c])","([9b, 9c], [5d, 8b, 7a])","([1b, 9a], [6d, 1d, 3d])","([5c, 6c], [2b, 1a, 7b])",player7
9,"([6c, 2d], [5b, 8d, 7b])","([], [])","([], [])","([4b, 8b], [5d, 3d, 2b])","([], [])","([9a, 1a], [2a, 5c, 3a])","([1c, 2c], [7c, 4a, 9b])",player6


In [38]:
temp_xian,zhuang,point_z

({'player1': 8,
  'player3': 3,
  'player4': 8,
  'player5': 6,
  'player6': 7,
  'player7': 0},
 'player2',
 5)